# Build a graph with networkx and visualize with Matplotlib and Cytoscape

Its pretty hacking to get the build environment together, but it works!

Maybe write a little bash script to do the env setup and bootstrap with that?

In [ ]:
IN_COLAB = True
try:
  import google.colab
except:
  IN_COLAB = False

In [ ]:
!ls -l

In [ ]:
if IN_COLAB:
    !git clone https://github.com/LLNL/Wintap-Analytics.git
    !ln -s  Wintap-Analytics/notebooks/wintapgraph.py .
    from google.colab import output
    output.enable_custom_widget_manager()

In [ ]:
!pip install magic-duckdb duckdb==0.10.1 networkx ipycytoscape ipywidgets matplotlib

In [ ]:
if IN_COLAB:
  !gdown 1ELBBx3p2ngVSIyf7Neu0F50vd-qf9XDO
else:
  !curl -O https://gdo168.llnl.gov/data/workshop/ACME-workshop-20231109-20231111.db

In [ ]:
# Import packages used in notebooks
#from wintappy.datautils import rawutil as ru
#from wintappy.datautils import stdview_duckdb as svd

#import os
#import altair as alt
import duckdb
import wintapgraph as wg
import networkx as nx
%load_ext magic_duckdb

In [ ]:
# Initialize an in-memory db. Save reference in a variable and then set magic-duckdb environment. Result is ability to use the same DB instance from python code and %dql/%%dql magics.
# Also create views for every top-level type found in the current dataset.
con = duckdb.connect('ACME-workshop-20231109-20231111.db')
%dql -co con
# Display the list of tables/views
%dql show tables

In [ ]:
style=[
        {
             'selector': 'node',
             'style': {
                 'font-family': 'helvetica',
                 'font-size': '20px',
                 'label': 'data(label)',
                 'text-wrap':'wrap'
             }
        },
        {
             'selector': 'edge',
             'style': {
                 'font-family': 'helvetica',
                 'font-size': '20px'
             }
        },
               {
             'selector': 'node[degree>0]',
             'style': {
                 'width': '100px',
                 'height': '100px'
             }
        },
        {
             'selector': 'node[degree>1]',
             'style': {
                 'width': '150px',
                 'height': '150px'
             }
        },
        {
             'selector': 'node[degree>2]',
             'style': {
                 'width': '100px',
                 'height': '300px'
             }
        }

    ]


In [ ]:
import ipycytoscape
import ipywidgets as widgets
import networkx as nx

netg, seed_processes = wg.build_graph(con, ["ssh.exe","putty"])
undirected = ipycytoscape.CytoscapeWidget(cytoscape_layout={'name': 'breadthfirst'},cytoscape_style=style)
undirected.graph.add_graph_from_networkx(netg)

display(undirected)

In [ ]:
from matplotlib import pyplot as plt
print('Drawing graph')
# Labels
label_dict = {n: netg.nodes()[n]['label'] for n in netg.nodes}
# Colors
# TODO add support for colors based on attributes in the graph.
#cmap = matplotlib.colors.ListedColormap(['dodgerblue', 'lightgray', 'darkorange'])
node_types_map = {'Root': 0, 'Parent': 1, 'Seed': 2}

plt.figure(figsize=(20, 14))
# Kamada Kawai produces reasonable layouts and they are consistent. Spring has randomness which is annoying for repeatability.
nx.draw_kamada_kawai(netg, with_labels=True, labels=label_dict) #,node_color=node_types,cmap=cmap)


Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
# Only use this if you want to disable 3rd party widgets
if False:
  from google.colab import output
  output.disable_custom_widget_manager()